# Yes24 베스트셀러 정보 저장하기

http://www.yes24.com/Main/default.aspx


In [1]:
# 관련 모듈 임포트
# 임포트
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# url => html 문자열화
url = 'http://www.yes24.com/Main/default.aspx'
response = requests.get(url)
html_str= response.text
with open('output/yes24_best.html','w') as f:
    f.write(html_str)

In [3]:
ls output

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 582F-D8CD

 C:\workspace\scrap\output 디렉터리

2021-09-10  오후 03:00    <DIR>          .
2021-09-10  오후 03:00    <DIR>          ..
2021-09-10  오후 02:28            12,432 color_info.csv
2021-09-10  오후 01:57           217,139 colors_names.html
2021-09-10  오전 10:47            15,041 google1.html
2021-09-10  오전 10:47            15,057 google2.html
2021-09-10  오전 11:24            11,936 warandpeace.html
2021-09-10  오후 03:00           566,652 yes24_best.html
               6개 파일             838,257 바이트
               2개 디렉터리  378,315,304,960 바이트 남음


In [4]:
# BeautifulSoup 객체화 
soup = BeautifulSoup(html_str, 'html.parser')
soup


<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="dpr, width, viewport-width, rtt, downlink, ect, UA, UA-Platform, UA-Arch, UA-Model, UA-Mobile, UA-Full-Version" http-equiv="Accept-CH"/>
<meta content="86400" http-equiv="Accept-CH-Lifetime"/>
<meta content="width=1170" name="viewport"/>
<title>YES24</title>
<meta content="YES24" name="title"/>
<meta content="YES24는 대한민국 1위 인터넷 온라인 서점 입니다. 국내 최대의 도서정보를 보유하고 있으며, 음반, DVD, 공연, 영화까지 다양한 문화 콘텐츠 및 서비스를 제공합니다." name="description"/>
<meta content="인터넷 서점, 온라인 쇼핑, 상품 추천, 쇼핑몰, 상품 검색, 도서 정보, 국내도서, 외국도서, 전자책, eBook, 이북, 크레마, 공연, 콘서트, 뮤지컬, 영화, 음반, 예매, DVD, 블루레이, 예스24, YES24, 교보문고, 알라딘" name="keywords"/>
<meta content="YES24" property="og:title"/>
<meta content="YES24는 대한민국 1위 인터넷 온라인 서점 입니다. 국내 최대의 도서정보를 보유하고 있으며, 음반, DVD, 공연, 영화까지 다양한 문화 콘텐츠 및 서비스를 제공합니다." property="og:description"/>
<meta content="//image.yes24.com/sys

In [6]:
# 추출하고자 하는 책 정보 박스 필터링  
# find_all(class_='클래스명') => 리스트 
# .tp02
li_list = soup.find_all(class_='tp02')
len(li_list)

10

In [7]:
# 5위
li_list[4]

<li class="tp02">
<strong class="icon rnk_5">5위</strong>
<em class="rnk_chg chg_up">1</em>
<span class="rnk_img"><img alt="양순이네 떡집" border="0" src="http://image.yes24.com/Goods/103303100/S" width="40"/></span>
<span class="rnk_info">
<strong>양순이네 떡집</strong>
<em>김리리 글/김이랑 그림</em>
</span>
<a class="rnk_lnk" href="/Product/Goods/103303100" onclick="setWcode('010_006');" target="_self" title="양순이네 떡집 김리리 글/김이랑 그림 비룡소">'양순이네 떡집' 상품상세로 이동</a>
</li>

In [18]:
print('순위 =>',li_list[4].select('strong')[0].text)
print('책제목 =>',li_list[4].select('strong')[1].text)
print('저자 =>',li_list[4].select('em')[1].text)
print('표지 =>',li_list[4].select_one('img')['src'].replace('S','XL'))
base_url = 'http://www.yes24.com'
print('URL =>',base_url + li_list[4].select_one('a')['href'])

순위 => 5위
책제목 => 양순이네 떡집
저자 => 김리리 글/김이랑 그림
표지 => http://image.yes24.com/Goods/103303100/XL
URL => http://www.yes24.com/Product/Goods/103303100


In [24]:
# 1~10위의 책 정보를 리스트에 저장하기 => 2차원 구조
book_list = []
base_url = 'http://www.yes24.com'
for li in li_list:
    rank = li.select('strong')[0].text
    title = li.select('strong')[1].text
    writer = li.select('em')[1].text
    poster = li.select_one('img')['src'].replace('S','XL')
    url = base_url + li_list[4].select_one('a')['href']
    # 1차원 리스트 구조로 생성한 후 다시 리스트에 저장 => 2차원
    book_list.append([rank, title, writer,poster, url])

len(book_list)

10

In [25]:
# 데이터프레임 형태로 저장
field_list = ['rank', 'title', 'writer', 'poster', 'URL']
df = pd.DataFrame(book_list, columns=field_list)
df.head()

,rank,title,writer,poster,URL
0,1위,달러구트 꿈 백화점 2,이미예 저,http://image.yes24.com/Goods/102789938/XL,http://www.yes24.com/Product/Goods/103303100
1,2위,달러구트 꿈 백화점,이미예 저,http://image.yes24.com/Goods/91065309/XL,http://www.yes24.com/Product/Goods/103303100
2,3위,"오늘 밤, 세계에서 이 사랑이 사라진다 해도",이치조 미사키 저/권영주 역,http://image.yes24.com/Goods/102360203/XL,http://www.yes24.com/Product/Goods/103303100
3,4위,작별하지 않는다,한강 저,http://image.yes24.com/Goods/103495056/XL,http://www.yes24.com/Product/Goods/103303100
4,5위,양순이네 떡집,김리리 글/김이랑 그림,http://image.yes24.com/Goods/103303100/XL,http://www.yes24.com/Product/Goods/103303100


In [26]:
# CSV 파일로 저장 
df.to_csv('output/book_bestseller.csv', index=False)

In [27]:
ls output

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 582F-D8CD

 C:\workspace\scrap\output 디렉터리

2021-09-10  오후 03:26    <DIR>          .
2021-09-10  오후 03:26    <DIR>          ..
2021-09-10  오후 03:26             1,662 book_bestseller.csv
2021-09-10  오후 02:28            12,432 color_info.csv
2021-09-10  오후 01:57           217,139 colors_names.html
2021-09-10  오전 10:47            15,041 google1.html
2021-09-10  오전 10:47            15,057 google2.html
2021-09-10  오전 11:24            11,936 warandpeace.html
2021-09-10  오후 03:00           566,652 yes24_best.html
               7개 파일             839,919 바이트
               2개 디렉터리  378,286,587,904 바이트 남음
